In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import string
import os
import datetime
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
from scipy.linalg import svd
from sklearn import ensemble
import scipy.linalg as la
import scipy.sparse.linalg as sla
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from time import time
from sklearn import metrics

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
X_all = pd.read_csv('../input/project2/X_cleaned.csv')
Y_all = pd.read_csv('../input/project2/Y_cleaned.csv')
Date_index = pd.read_csv('../input/project2/Date_list.csv')
Date_index['DATE']=pd.to_datetime(Date_index['DATE'])

In [ ]:
start_date = pd.to_datetime('1957-03-01')
end_date =  start_date + pd.offsets.DateOffset(years=18)
ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
ytrain = Y_all[ind_training]
xtrain = X_all[ind_training]

In [ ]:
start_date =  end_date
end_date = start_date + pd.offsets.DateOffset(years=12)
ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
ytest = Y_all[ind_vali]
xtest= X_all[ind_vali]

In [ ]:
start_date =  end_date
end_date = start_date + pd.offsets.DateOffset(years=30)
ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
#print(type(ind_test))
yoos = Y_all[ind_test]
xoos= X_all[ind_test]

In [ ]:
#######Monthly Demean %%%
ytrain_demean= ytrain - np.mean(ytrain);
ytest_demean=ytest-np.mean(ytest);
mtrain=np.mean(ytrain);
mtest=np.mean(ytest);

sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
for i in range(len(xtrain.columns)):
    s=np.std(xtrain.iloc[:,[i]])
    colnames = xtrain.columns
    if s.values > 0:
        colname = colnames[i]
        xtrain.loc[:,colname]=xtrain[colname]/s.values
        xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
        xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
        sd[i]=s.values
    #### Start to Train %%%
    #model fitting

In [ ]:
def fw2(x):
    maximum = max(map(max,x))
    p = np.where(x == maximum)
    return p

In [ ]:
###Number of predictors
nump = xtrain.shape[1]
######num of predictors to sample
lamv = np.arange(10,nump,50)
###try
#lamv = [100]

##num of trees
ne=100
####maximum of split

lamc = [2,4,8,16,32]
####try
#lamc = [8];
r = np.zeros((len(lamv),len(lamc),3))

for n1 in range(len(lamv)):
    nf=lamv[n1]
    for n2 in range(len(lamc)):
        nc = lamc[n2]
        clf = ensemble.RandomForestRegressor(n_estimators = ne,max_features = nf,max_depth= nc)

        clffit = clf.fit(xtrain,np.ravel(ytrain))
        yhatbig1 = clffit.predict(xtest)
        r[n1,n2,0]=1 - np.linalg.norm(yhatbig1-ytest['RET'])**2 / np.linalg.norm((ytest['RET']-mtrain.values))**2
        yhatbig1= clffit.predict(xoos)
        r[n1,n2,1]=1-np.linalg.norm((yhatbig1-yoos['RET']))**2/np.linalg.norm((yoos['RET']-mtrain.values))**2
        yhatbig1= clffit.predict(xtrain)
        r[n1,n2,2]=1-np.linalg.norm((yhatbig1-ytrain['RET']))**2/np.linalg.norm((ytrain['RET']-mtrain.values))**2

fw_2 = fw2(r[:,:,0])
r2_oos=r[fw_2[0][0],fw_2[1][0],1]
r2_is=r[fw_2[0][0],fw_2[1][0],2]
print('RF R2 : '+np.str0(r2_oos))


In [ ]:
for i in range(len(X_all.columns)):
    xtrain_1=xtrain.drop(columns=[X_all.columns[i]])
    xtest_1=xtest.drop(columns=[X_all.columns[i]])
    xoos_1= xoos.drop(columns=[X_all.columns[i]])
    
    ###Number of predictors
    nump = xtrain_1.shape[1]
    ######num of predictors to sample
    lamv = np.arange(10,nump,50)
    ###try
    #lamv = [100]

    ##num of trees
    ne=100
    ####maximum of split

    lamc = [2,4,8,16,32]
    ####try
    #lamc = [8];
    r = np.zeros((len(lamv),len(lamc),3))

    for n1 in range(len(lamv)):
        nf=lamv[n1]
        for n2 in range(len(lamc)):
            nc = lamc[n2]
            clf = ensemble.RandomForestRegressor(n_estimators = ne,max_features = nf,max_depth= nc)

            clffit = clf.fit(xtrain_1,np.ravel(ytrain))
            yhatbig1 = clffit.predict(xtest_1)
            r[n1,n2,0]=1 - np.linalg.norm(yhatbig1-ytest['RET'])**2 / np.linalg.norm((ytest['RET']-mtrain.values))**2
            yhatbig1= clffit.predict(xoos_1)
            r[n1,n2,1]=1-np.linalg.norm((yhatbig1-yoos['RET']))**2/np.linalg.norm((yoos['RET']-mtrain.values))**2
            yhatbig1= clffit.predict(xtrain_1)
            r[n1,n2,2]=1-np.linalg.norm((yhatbig1-ytrain['RET']))**2/np.linalg.norm((ytrain['RET']-mtrain.values))**2

    fw_2 = fw2(r[:,:,0])
    r2_oos_1=r[fw_2[0][0],fw_2[1][0],1]
    r2_is_1=r[fw_2[0][0],fw_2[1][0],2]
    d=r2_oos-r2_oos_1
    print(X_all.columns[i]+':'+np.str0(d))

        

<!--
beta:0.029227020177641894
betasq:0.0760829365645983
chmom:0.01926140345542926
dolvol:-0.03608237769034006
idiovol:-0.048767954736835595
indmom:-0.003641258117789592
mom1m:0.02669423380766922
mom6m:0.05441371398067152
mom12m:0.0005489330941026438
mom36m:-0.00503827854663097
pricedelay:0.047043385087462086
turn:-0.021599761408751084
absacc:-0.01035980692004923
acc:0.027893754854211528
age:0.10377845482227399
agr:0.03917949358500428
cashdebt:0.06501226959723794
cashpr:0.012597490220743968
cfp:0.019567109408422434
cfp_ia:0.01596782659532625
chatoia:-0.058749088754847856
chcsho:0.06248248719981264
chempia:0.004264217968765527
chinv:0.009298144964421384
chpmia:0.05156191982383951
convind:0.013648186899199999
currat:0.15715009316430062
depr:0.04816170052441193
divi:0.04299246238447196
divo:0.052028077613764934
dy:-0.0054784649061097035
egr:0.09048540559299667
ep:0.00836078992044098
gma:-0.011308012267343237
grcapx:0.013316311208009246
grltnoa:0.0196532608753075
herf:0.015571488533427269
hire:0.07372218174202061
invest:-0.015174370365575784
lev:0.07585975513994003
lgr:-0.019568063312263906
mve_ia:0.06318789210059284
operprof:-0.061236108231405684
orgcap:0.00653854670320686
pchcapx_ia:0.1717634971427877
pchcurrat:0.025431084561027717
pchdepr:0.020683425819052914
pchgm_pchsale:0.003022032165736377
pchquick:0.03612448967634063
pchsale_pchinvt:-0.056254991845593905
pchsale_pchrect:-0.054743732655047506
pchsale_pchxsga:0.06615908509864532
pchsaleinv:-0.04908425499449387
pctacc:0.042881893831892004
ps:-0.009579618859268946
quick:0.03563523965716553
rd:-0.01611751056699795
rd_mve:-0.005532613552249099
rd_sale:-0.03356283732852239
realestate:-0.004785266885392714
roic:0.07115423698835666
salecash:-0.00980512123837518
saleinv:-0.014041229246911868
salerec:0.03545924710162973
secured:0.03406607676964457
securedind:0.0349718549737843
sgr:0.036378012543789806
sin:-0.03314444100740288
sp:0.03947604531904925
tang:-0.06037105667268916
tb:-0.007526825534889703
aeavol:-0.009239045431428172
cash:-0.07063354753531681
chtx:0.031558900836135884
cinvest:-0.0029453777778121193
ear:0.056845777849523715
nincr:0.032874842449877706
roaq:0.006271384868989038
roavol:0.0011673053921730236
roeq:0.023403922529807275
rsup:0.060899627159751946
stdacc:-0.051937594076731264
stdcf:-0.05345967912475169
ms:0.04553522549059785
baspread:0.004767782704576229
ill:0.03631967490735488
maxret:0.02620839765683436
retvol:-0.021583696172512434
std_dolvol:-0.008652650033020404
std_turn:-0.03169950736719618
zerotrade:-0.0004310199442363327
bm:-0.004485533348450588
bm_ia:0.013903133882749996
sic2_10:0.017751690196417647
sic2_13:0.09484817414722668
sic2_15:0.044093131876905334
sic2_17:0.0447916667670587
sic2_20:0.019278951553208135
sic2_22:-0.007349424206752442
sic2_23:-0.0262330657114187
sic2_24:-0.03214185009070647
sic2_25:0.02066260574576151
sic2_26:0.034143320788051934
sic2_27:0.03835294796106714
sic2_28:0.06423122840019446
sic2_29:-0.023095514234823034
sic2_30:0.053323680477443514
sic2_32:-0.023180794633780977
sic2_33:0.011730115703527222
sic2_34:-0.012186811226454708
sic2_35:-0.036829144823282256
sic2_36:0.03898851363310274
sic2_37:-0.004767997139897506
sic2_38:-0.03799596711021702
sic2_39:0.024383966795524126
sic2_40:0.03838971298569893
sic2_41:0.2250304018924869
sic2_42:0.021515734691873067
sic2_44:0.014452860818104263
sic2_45:0.018638157403672695
sic2_47:0.08182080038717698
sic2_48:0.07050294008732894
sic2_49:0.023291220093931475
sic2_50:0.014959823392025617
sic2_51:0.024926050217622064
sic2_52:-0.019527757722915373
sic2_53:-0.004776893007423899
sic2_54:0.015693662673298192
sic2_56:0.04574846140741662
sic2_57:0.05885801864369711
sic2_58:-0.04298649644175567
sic2_59:0.046626860438435225
sic2_60:-0.031305105760051744
sic2_61:-0.014212826720917171
sic2_62:0.023338997708871823
sic2_63:-0.03403597116376944
sic2_64:0.024476203687678444
sic2_65:-0.023189517054938102
sic2_67:-0.021460287197457406
sic2_70:-0.04912905570749371
sic2_72:0.0043986664934045905

In [ ]:
# kaggle cloud is interropted so we run the code in another process
for i in range(len(X_all.columns)-141):
    xtrain_1=xtrain.drop(columns=[X_all.columns[i+141]])
    xtest_1=xtest.drop(columns=[X_all.columns[i+141]])
    xoos_1= xoos.drop(columns=[X_all.columns[i+141]])
    
    ###Number of predictors
    nump = xtrain_1.shape[1]
    ######num of predictors to sample
    lamv = np.arange(10,nump,50)
    ###try
    #lamv = [100]

    ##num of trees
    ne=100
    ####maximum of split

    lamc = [2,4,8,16,32]
    ####try
    #lamc = [8];
    r = np.zeros((len(lamv),len(lamc),3))

    for n1 in range(len(lamv)):
        nf=lamv[n1]
        for n2 in range(len(lamc)):
            nc = lamc[n2]
            clf = ensemble.RandomForestRegressor(n_estimators = ne,max_features = nf,max_depth= nc)

            clffit = clf.fit(xtrain_1,np.ravel(ytrain))
            yhatbig1 = clffit.predict(xtest_1)
            r[n1,n2,0]=1 - np.linalg.norm(yhatbig1-ytest['RET'])**2 / np.linalg.norm((ytest['RET']-mtrain.values))**2
            yhatbig1= clffit.predict(xoos_1)
            r[n1,n2,1]=1-np.linalg.norm((yhatbig1-yoos['RET']))**2/np.linalg.norm((yoos['RET']-mtrain.values))**2
            yhatbig1= clffit.predict(xtrain_1)
            r[n1,n2,2]=1-np.linalg.norm((yhatbig1-ytrain['RET']))**2/np.linalg.norm((ytrain['RET']-mtrain.values))**2

    fw_2 = fw2(r[:,:,0])
    r2_oos_1=r[fw_2[0][0],fw_2[1][0],1]
    r2_is_1=r[fw_2[0][0],fw_2[1][0],2]
    d=r2_oos-r2_oos_1
    print(X_all.columns[i+141]+':'+np.str0(d))


<!--
sic2_73:0.006055869428575056
sic2_75:-0.08242972297501239
sic2_78:0.008387160021703144
sic2_79:0.021940562981372258
sic2_80:-0.030618373120196107
sic2_87:-0.040437031598137496
sic2_99:-0.0448481341271294